In [4]:
# install packages
from bs4 import BeautifulSoup
import requests
import smtplib
import time
import datetime
import pandas as pd

In [30]:
# determine the data where you will scrape
baseurl = 'https://www.trendyol.com/'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36"} 
# determine how much data do you want to pull out 
productlinks = []
for x in range(1,200):
    r = requests.get(f'https://www.trendyol.com/sr/?fl=encoksatanurunler&sst=MOST_RATED&pi={x}', headers=headers)
    soup1 = BeautifulSoup(r.content, 'html.parser')
    productlist = soup1.find_all('div', class_="p-card-wrppr")
    for item in productlist:
        for link in item.find_all('a', href=True):
            productlinks.append(baseurl + link['href'])

# check for errors using test_link            
#test_link = 'https://www.trendyol.com/trendyolmilla/siyah-yuksek-bel-toparlayici-orme-tayt-twoaw20ta0087-p-31515569?boutiqueId=605726&merchantId=968'

# pull out specific data
clotheslist = []
for link in productlinks:

    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    seller = soup.find('a', class_="merchant-text").text.strip()
    try:    
        brand = soup.find('h1', class_="pr-new-br").a.text.strip()
    except:
        brand = "no brand"
    product = soup.find('h1', class_="pr-new-br").span.text.strip()
    price = soup.find('span', class_="prc-dsc").text.strip()
    rating = soup.find(class_="rvw-cnt-tx").text.strip()
    favorite = soup.find(class_="fv-dt").text.strip()
    try:
        category = soup.find(class_="category-gender-desc").text
    except:
        category = 'no category'
    
    clothes = {
        'seller' : seller, 'brand' : brand, 'product' : product, 'favorite' : favorite, 'price' : price, 'rating' : rating, 'category' : category
    }
    
    clotheslist.append(clothes)

# write to excel 

df = pd.DataFrame(clotheslist)
df.to_csv('5_17_2022TYwebsc.csv', encoding = "UTF-8", index=False)